In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from spellchecker import SpellChecker
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

import pandas as pd 
import nltk as nltk
import  xgboost, numpy, string
import datetime as dt
#from keras.preprocessing import text, sequence
#from keras import layers, models, optimizers

##### Functions

In [2]:
# Timer to check execution timing for each function call # 
def timer(start_time=None):
    if not start_time:
        start_time = dt.datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((dt.datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

# Spelling checker # 
def spell_correct(array):
    spell = SpellChecker()
    for i in range(len(array)):
        array[i] = spell.correction(array[i])
    return array
    
def stem(array):
    stemmer = nltk.PorterStemmer()
    return [stemmer.stem(w) for w in array]

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # Fit the training dataset onto classifier #
    classifier.fit(feature_vector_train, label)
    # Predict the labels on validation dataset #
    predictions = classifier.predict(feature_vector_valid)
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    return metrics.accuracy_score(predictions, valid_y)

##### Data Prep 

In [3]:
# Data Prep# 
df = pd.read_csv("data/cleaned_hotelreviews.csv")

# Drop rows with null comments # 
df = df.dropna(how='any', axis=0)

# Make words case-insensitive # 
df = df.apply(lambda x: x.astype(str).str.lower())

# Remove punctuations if any # 
df["words_only"] = df['reviews'].str.replace('[^\w\s]','')

# Tokenization with NLTK # 
start_time = timer(None)
df['tokenized'] = df['words_only'].apply(nltk.word_tokenize)
print("\nTokenizer: ",end="")
timer(start_time)

# Spelling checker # : Replace incorrect words with correct words 
#start_time = timer(None)
#df['corrected'] = df['tokenized'].apply(spell_correct)
#print("\nSpelling Correction: ",end="")
#timer(start_time)

# Stemming with NLTK # 
start_time = timer(None)
df['stemmed'] = df['tokenized'].apply(stem)
print("\nStemming: ",end="")
timer(start_time)

# Turn arrays for each row in df['stemmed'] into a string #: Needed to run SkLearn Lib
df['stemmed'] = df['stemmed'].apply(" ".join)

# Train - Test Split # 
start_time = timer(None)
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['stemmed'], df['class'])
print("\nTrain-Test Split: ",end="")
timer(start_time)

# Label encode target variable to run ML models # 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


Tokenizer: 
 Time taken: 0 hours 3 minutes and 27.75 seconds.

Stemming: 
 Time taken: 0 hours 11 minutes and 7.13 seconds.

Train-Test Split: 
 Time taken: 0 hours 0 minutes and 0.49 seconds.


##### Count vectorisation : Create vectors as features 
    # Every row represents a review 
    # Every column represents a term from the corpus 
    # Every cell represents the frequency count of the particular term in the particular review 

In [4]:
start_time = timer(None)

# Create count vectoriser object # 
count_vector = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vector.fit(df['stemmed'])

# Transform training and validation data # 
xtrain_count = count_vector.transform(train_x)
xvalid_count = count_vector.transform(valid_x)

print("Count Vectors:", end="")
timer(start_time)

Count Vectors:
 Time taken: 0 hours 1 minutes and 7.11 seconds.


In [12]:
print(xtrain_count)

  (0, 2417)	1
  (0, 11832)	1
  (0, 24284)	1
  (0, 28487)	1
  (0, 34514)	1
  (0, 48410)	1
  (0, 49270)	1
  (0, 49582)	1
  (1, 1506)	1
  (1, 17408)	1
  (2, 139)	1
  (2, 2033)	2
  (2, 7887)	1
  (2, 12141)	1
  (2, 17171)	1
  (2, 19718)	1
  (2, 22770)	1
  (2, 23110)	1
  (2, 23428)	1
  (2, 23868)	1
  (2, 28487)	1
  (2, 32219)	1
  (2, 33695)	1
  (2, 36347)	2
  (2, 37674)	1
  :	:
  (625579, 16600)	1
  (625579, 20207)	1
  (625579, 21312)	1
  (625579, 24284)	1
  (625579, 28577)	1
  (625579, 28890)	1
  (625579, 29905)	1
  (625579, 31072)	1
  (625579, 32641)	1
  (625579, 36634)	1
  (625579, 39875)	1
  (625579, 42031)	1
  (625579, 44912)	1
  (625579, 46370)	1
  (625579, 46647)	1
  (625579, 49290)	3
  (625579, 50050)	1
  (625579, 50449)	1
  (625579, 50746)	1
  (625579, 53999)	1
  (625579, 54472)	1
  (625579, 54633)	1
  (625579, 55170)	1
  (625579, 55314)	1
  (625580, 17408)	1


### Convert to TF-IDF Vectors 


In [5]:
start_time = timer(None)

# Word Level TF-IDF #: Matrix represents tf-idf scores of every term in each review 
tfidf_word = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_word.fit(df['stemmed'])
xtrain_tfidf_word = tfidf_word.transform(train_x)
xvalid_tfidf_word = tfidf_word.transform(valid_x)

print("Word Level TF-IDF", end="")
timer(start_time)

Word Level TF-IDF
 Time taken: 0 hours 1 minutes and 9.53 seconds.


In [6]:
start_time = timer(None)

# Unigram Level TF-IDF #: Matrix represents tf-idf scores of unigram (all terms are separate)
tfidf_unigram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1), max_features=5000)
tfidf_unigram.fit(df['stemmed'])
xtrain_tfidf_unigram = tfidf_unigram.transform(train_x)
xvalid_tfidf_unigram = tfidf_unigram.transform(valid_x)

# Bigram Level TF-IDF #: Terms are grouped together by twos 
tfidf_bigram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,2), max_features=5000)
tfidf_bigram.fit(df['stemmed'])
xtrain_tfidf_bigram = tfidf_bigram.transform(train_x)
xvalid_tfidf_bigram = tfidf_bigram.transform(valid_x)

# Trigram Level TF-IDF #: Terms are grouped together in threes 
tfidf_trigram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(3,3), max_features=5000)
tfidf_trigram.fit(df['stemmed'])
xtrain_tfidf_trigram = tfidf_trigram.transform(train_x)
xvalid_tfidf_trigram = tfidf_trigram.transform(valid_x)

print("Unigram, Bigram, Trigram TF-IDF:", end="")
timer(start_time)

Unigram, Bigram, Trigram TF-IDF:
 Time taken: 0 hours 5 minutes and 8.71 seconds.


In [15]:
print(xtrain_tfidf_unigram)

  (0, 4411)	0.4576933239103641
  (0, 4388)	0.2944212060537047
  (0, 4310)	0.28229518398521697
  (0, 2962)	0.21183835045414057
  (0, 2474)	0.32590584463074085
  (0, 2133)	0.2494028458757174
  (0, 1003)	0.42973576119395684
  (0, 194)	0.47558142678730936
  (1, 1501)	0.5471795605489724
  (1, 130)	0.8370152498714902
  (2, 4786)	0.07618026563249647
  (2, 4743)	0.08732790547926773
  (2, 4696)	0.1718860077237478
  (2, 4554)	0.29825644750067865
  (2, 4389)	0.062208280070648436
  (2, 4223)	0.22665233453579417
  (2, 4120)	0.08506546618465322
  (2, 3252)	0.20496306703249728
  (2, 3135)	0.5382438336890729
  (2, 2895)	0.1304426862639404
  (2, 2781)	0.19644653108750007
  (2, 2474)	0.14711673786505555
  (2, 2101)	0.09636996916686696
  (2, 2052)	0.26294155882917025
  (2, 2026)	0.1183182355573233
  :	:
  (625579, 4495)	0.2416933949684209
  (625579, 4458)	0.08474856642238468
  (625579, 4389)	0.19310606761460686
  (625579, 4120)	0.08801967436953376
  (625579, 4097)	0.2601575017524426
  (625579, 3936)	0.18

In [16]:
print(type(xtrain_tfidf_unigram))

<class 'scipy.sparse.csr.csr_matrix'>


In [7]:
start_time = timer(None)

# Character Level TF-IDF #: Matrix represents tf-idf scores of character level uni, bi & tri-gram of all reviews
tfidf_char = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(1,3), max_features=5000)
tfidf_char.fit(df['stemmed'])
xtrain_tfidf_char =  tfidf_char.transform(train_x) 
xvalid_tfidf_char =  tfidf_char.transform(valid_x) 

print("Character Level TF-IDF:", end="")
timer(start_time)

Character Level TF-IDF:
 Time taken: 0 hours 8 minutes and 49.03 seconds.


## XGBoost (Boosting)

In [8]:
# XGB Model # 
model = xgboost.XGBClassifier(max_depth=7,
                           min_child_weight=1,
                           learning_rate=0.2,
                           n_estimators=500,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=1,
                           seed=1,
                           missing=None,
                           tree_method='exact',
                           nthread=4)

# Params for hyperparameter grid search # 
params = {
        'max_depth': [5,7],
        'min_child_weight': [1, 5],
        'gamma': [0.5, 1]
        }

In [9]:
start_time = timer(None)
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)
timer(start_time)

start_time = timer(None)
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_word.tocsc(), train_y, xvalid_tfidf_word.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)
timer(start_time)

start_time = timer(None)
# Extereme Gradient Boosting on Unigram Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_unigram.tocsc(), train_y, xvalid_tfidf_unigram.tocsc())
print ("Xgb, Unigram TF-IDF: ", accuracy)
timer(start_time)

start_time = timer(None)
# Extereme Gradient Boosting on Bigram Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_bigram.tocsc(), train_y, xvalid_tfidf_bigram.tocsc())
print ("Xgb, Bigram TF-IDF: ", accuracy)
timer(start_time)

start_time = timer(None)
# Extereme Gradient Boosting on Trigram Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_trigram.tocsc(), train_y, xvalid_tfidf_trigram.tocsc())
print ("Xgb, Trigram TF-IDF: ", accuracy)
timer(start_time)

start_time = timer(None)
# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_char.tocsc(), train_y, xvalid_tfidf_char.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)
timer(start_time)

Xgb, Count Vectors:  0.9293472787696557

 Time taken: 0 hours 2 minutes and 6.38 seconds.
Xgb, WordLevel TF-IDF:  0.9317450498017044

 Time taken: 0 hours 2 minutes and 50.55 seconds.
Xgb, Unigram TF-IDF:  0.9317450498017044

 Time taken: 0 hours 2 minutes and 48.8 seconds.
Xgb, Bigram TF-IDF:  0.8570784598637107

 Time taken: 0 hours 1 minutes and 26.16 seconds.
Xgb, Trigram TF-IDF:  0.7381394255899716

 Time taken: 0 hours 0 minutes and 39.41 seconds.
Xgb, CharLevel Vectors:  0.936516614155481

 Time taken: 0 hours 20 minutes and 40.38 seconds.


## Hyperparameter Grid Search with 3 Fold Validation

In [10]:
start_time = timer(None)

# 3 fold validation with hyperparameter grid search #
folds = 3
param_comb = 3

kf = KFold(n_splits=folds, shuffle = False, random_state = None)

grid = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', n_jobs=3, cv=kf.split(xtrain_count,train_y), 
                    verbose=3 )

start_time = timer(None)

grid.fit(xtrain_count,train_y)

print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_)
print('\n Best parameters:')
print(grid.best_params_)

timer(start_time)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  24 out of  24 | elapsed: 78.9min finished



 Best estimator:
XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.5, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.2, max_delta_step=0, max_depth=7,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=1, reg_alpha=0,
              reg_lambda=0, scale_pos_weight=1, seed=1, silent=True,
              subsample=1, tree_method='exact', validate_parameters=False,
              verbosity=None)

 Best score:
0.9425094432215813

 Best parameters:
{'gamma': 0.5, 'max_depth': 7, 'min_child_weight': 5}

 Time taken: 1 hours 26 minutes and 20.57 seconds.


### LDA Model 

In [17]:
# Train LDA Model # 
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vector.get_feature_names()

# View Topic Models # 
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

topic_summaries

['to door the call no hour access it up wall',
 'vegetarian choic option at least tast breakfast etc shop perhap',
 'interior theatreland mass charact style and a for with wa',
 'chair outlin terranc meter written premium deep a half the',
 'a and bar stay attract have better local for t',
 'some isn unit beat noisi upgrad my bread white bap',
 'tower grang closer fanciest weren either quadrupl slightli hill thought',
 'la hop rambla storag plastic catalunya ceram plaza goog spectacular',
 'headboard a includ bed with breakfast curtain gbp appoint english',
 'gener bit a expens to difficult car the hotel thi',
 'bedsid light use would been have a olllld menu frontag',
 'a bit the from far to but expens citi center',
 'slipper dirtili toothbrush toothpast bathrob no bathroom and a beauti',
 'a the and to in hotel wa room with beauti',
 'isol less certainli ac post until bedsid becom waterfront supermarket',
 'water bottl tap tabl just sad custom whi guest watch',
 'the and wa i a had fr

### NAIVE BAYES 

In [18]:
# Assumes indepedence among predictors # 
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Unigram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram)
print ("NB, Uni-Gram Vectors: ", accuracy)

# Naive Bayes on Bigram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram)
print ("NB, Bi-Gram Vectors: ", accuracy)

# Naive Bayes on Trigram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_trigram, train_y, xvalid_tfidf_trigram)
print ("NB, Tri-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_char, train_y, xvalid_tfidf_char)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.952
NB, WordLevel TF-IDF:  0.9
NB, Uni-Gram Vectors:  0.9
NB, Bi-Gram Vectors:  0.916
NB, Tri-Gram Vectors:  0.844
NB, CharLevel Vectors:  0.84


## Linear Classifier (Logistic Regression) 

In [19]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Unigram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram)
print ("LR, Uni-Gram Vectors: ", accuracy)

# Linear Classifier on Bigram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram)
print ("LR, Bi-Gram Vectors: ", accuracy)

# Linear Classifier on Trigram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_trigram, train_y, xvalid_tfidf_trigram)
print ("LR, Tri-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_char, train_y, xvalid_tfidf_char)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.956
LR, WordLevel TF-IDF:  0.932
LR, Uni-Gram Vectors:  0.932
LR, Bi-Gram Vectors:  0.884
LR, Tri-Gram Vectors:  0.832
LR, CharLevel Vectors:  0.94


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## SVM Model

In [20]:
start_time = timer(None)
# SVM on Count Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Level Vectors: ", accuracy)
timer(start_time)

SVM, Count Level Vectors:  0.82

 Time taken: 0 hours 0 minutes and 0.09 seconds.


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [21]:
# Supervised ML Algo that extracts best possible hyper-plane/ line that segregates the two classes #

# SVM on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("SVM, Word Level Vectors: ", accuracy)

# SVM on Unigram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram)
print ("SVM, Uni-Gram Vectors: ", accuracy)

# SVM on Bigram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram)
print ("SVM, Bi-Gram Vectors: ", accuracy)

# SVM on Trigram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_trigram, train_y, xvalid_tfidf_trigram)
print ("SVM, Tri-Gram Vectors: ", accuracy)
       
# SVM on Char Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_char, train_y, xvalid_tfidf_char)
print ("SVM, CharLevel Vectors: ", accuracy)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, Word Level Vectors:  0.816
SVM, Uni-Gram Vectors:  0.816
SVM, Bi-Gram Vectors:  0.816


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, Tri-Gram Vectors:  0.816
SVM, CharLevel Vectors:  0.816


## Random Forest (Bagging)

In [22]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_word, train_y, xvalid_tfidf_word)
print ("RF, Word Level Vectors: ", accuracy)

# RF on Unigram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_unigram, train_y, xvalid_tfidf_unigram)
print ("RF, Uni-gram TF-IDF: ", accuracy)

# RF on Bigram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_bigram, train_y, xvalid_tfidf_bigram)
print ("RF, Bi-gram TF-IDF: ", accuracy)

# RF on Trigram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_trigram, train_y, xvalid_tfidf_trigram)
print ("RF, Tri-gram TF-IDF: ", accuracy)

# RF on Char Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_char, train_y, xvalid_tfidf_char)
print ("RF, CharLevel TF-IDF: ", accuracy)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, Count Vectors:  0.952
RF, Word Level Vectors:  0.952
RF, Uni-gram TF-IDF:  0.944


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, Bi-gram TF-IDF:  0.944
RF, Tri-gram TF-IDF:  0.9
RF, CharLevel TF-IDF:  0.948


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


##  Feature Importance XGB 
###### Can't plot important features if we only have one train column 

In [ ]:
# Additional code for future reference # 
# Plot graph showing importance features, max = 50 features # 
importance = xgboost.XGBClassifier().feature_importances_ 
importance = pd.Series(importance, index=xtrain_count.columns)
importance.nlargest(50).plot(kind='barh')

# Select important features #
importance.sort_values(axis=0,ascending=False, inplace=True)
selected_features = importance.index[0:30].tolist()